In [5]:
import torch

model = torch.load('/home/maelic/Documents/RoboBreizh/RoboNLU/checkpoints/albert_04-10-16:46:07/model.pth')

# print number of parameters
total_params = sum(p.numel() for p in model.parameters())

print(f"Number of parameters: {total_params}")

Number of parameters: 25134903


In [22]:
import tflite_runtime.interpreter as tflite
from transformers import AutoTokenizer
import torch

interpreter = tflite.Interpreter(model_path="/home/maelic/Documents/RoboBreizh/RoboNLU/distilbert/tflite/distilbert_float32.tflite")
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

tf_lite_model = interpreter.get_signature_runner()

interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)

# dummy input

input_ids = torch.randint(0, 30522, (1, 32))
input_ids = input_ids.numpy().flatten()
input_ids = input_ids.astype("int64")
input_ids = input_ids.reshape(1, -1)
print(input_ids)
interpreter.set_tensor(input_details[0]['index'], input_ids)

interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])

ValueError: SignatureDef signature_key is None and model has 0 Signatures. None is only allowed when the model has 1 SignatureDef

In [2]:

# optimize transformer-based models with onnxruntime-tools
from onnx import version_converter, helper
import onnx
from onnxconverter_common import float16

input_model = '/home/maelic/Documents/robocup2023/NLP/nlp/checkpoints/distil_bert/distil_bert.onnx'
output_fp16 = '/home/maelic/Documents/robocup2023/NLP/nlp/checkpoints/distil_bert/distil_bert_fp16.onnx'
#optimized_model = optimizer.optimize_model(input_model, model_type='bert', num_heads=12, hidden_size=768)

model = onnx.load(input_model)

model_fp16 = float16.convert_float_to_float16(model)

onnx.save(model_fp16, output_fp16)

# A full list of supported adapters can be found here:
# https://github.com/onnx/onnx/blob/main/onnx/version_converter.py#L21
# Apply the version conversion on the original model
# onnx.checker.check_model(model_fp16)

# converted_model = version_converter.convert_version(model_fp16, 6)

# onnx.save(converted_model, "/home/maelic/Documents/robocup2023/NLP/nlp/checkpoints/bert/bert_fp16_v6.onnx")

# print(f"The model after conversion:\n{converted_model}")

In [ ]:
from onnxruntime.quantization.shape_inference import quant_pre_process
input_model = '/home/maelic/Documents/robocup2023/NLP/nlp/checkpoints/mobile_bert/mobile_bert'
output_fp16 = '/home/maelic/Documents/robocup2023/NLP/nlp/checkpoints/mobile_bert/mobile_bert.opt.onnx'
quant_pre_process(input_model_path=input_model+'.onnx', output_model_path=input_model+".preprocessed.onnx")

In [7]:
from onnxruntime.quantization import quantize_dynamic, QuantType

model_fp32 = '/home/maelic/Documents/robocup2023/NLP/nlp/checkpoints/mobile_bert/mobile_bert.onnx'
model_quant = '/home/maelic/Documents/robocup2023/NLP/nlp/checkpoints/mobile_bert/mobile_bert.quant.onnx'
quantized_model = quantize_dynamic(model_fp32, model_quant, optimize_model=True)

Ignore MatMul due to non constant B: /[MatMul_196]
Ignore MatMul due to non constant B: /[MatMul_201]
Ignore MatMul due to non constant B: /[MatMul_266]
Ignore MatMul due to non constant B: /[MatMul_271]
Ignore MatMul due to non constant B: /[MatMul_336]
Ignore MatMul due to non constant B: /[MatMul_341]
Ignore MatMul due to non constant B: /[MatMul_406]
Ignore MatMul due to non constant B: /[MatMul_411]
Ignore MatMul due to non constant B: /[MatMul_476]
Ignore MatMul due to non constant B: /[MatMul_481]
Ignore MatMul due to non constant B: /[MatMul_546]
Ignore MatMul due to non constant B: /[MatMul_551]
Ignore MatMul due to non constant B: /[MatMul_616]
Ignore MatMul due to non constant B: /[MatMul_621]
Ignore MatMul due to non constant B: /[MatMul_686]
Ignore MatMul due to non constant B: /[MatMul_691]
Ignore MatMul due to non constant B: /[MatMul_756]
Ignore MatMul due to non constant B: /[MatMul_761]
Ignore MatMul due to non constant B: /[MatMul_826]
Ignore MatMul due to non consta

In [ ]:
import onnx
from onnxconverter_common import float16

model = onnx.load("/home/maelic/Documents/robocup2023/NLP/nlp/checkpoints/mobile_bert/mobile_bert.onnx")
model_fp16 = float16.convert_float_to_float16(model)
onnx.save(model_fp16, "/home/maelic/Documents/robocup2023/NLP/nlp/checkpoints/mobile_bert/mobile_bert_fp16.onnx")

In [ ]:
!onnx2tf -i mobile_bert/mobilebert.onnx -osd -rtpo Erf

In [ ]:
!saved_model_cli show --dir saved_model/ --tag_set serve --signature_def serving_default

In [ ]:
import numpy as np
from transformers import AutoTokenizer


def convert_input_file_to_dataloader(words,model_name,
                                        cls_token_segment_id=0,
                                        pad_token_segment_id=0,
                                        sequence_a_segment_id=0,
                                        mask_padding_with_zero=True):
    if model_name == 'bert':
        tokenizer_name = 'bert-base-uncased'
    elif model_name == 'mobile_bert':
        tokenizer_name = 'google/mobilebert-uncased'
    elif model_name == 'distil_bert':
        tokenizer_name = 'distilbert-base-uncased'
    
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    max_seq_len = 32
    pad_token_label_id = 0

    # Setting based on the current model type
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    unk_token = tokenizer.unk_token
    pad_token_id = tokenizer.pad_token_id

    tokens = []
    slot_label_mask = []
    pro_labels_ids = []
    for word in words:
        word_tokens = tokenizer.tokenize(word)
        if not word_tokens:
            word_tokens = [unk_token]  # For handling the bad-encoded word
        else:
            pro_label = 0
        tokens.extend(word_tokens)
        # Use the real label id for the first token of the word, and padding ids for the remaining tokens
        slot_label_mask.extend(
            [pad_token_label_id + 1] + [pad_token_label_id] * (len(word_tokens) - 1))
        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        pro_labels_ids.extend(
            [pro_label] + [pad_token_label_id] * (len(word_tokens) - 1))

    # Account for [CLS] and [SEP]
    special_tokens_count = 2
    if len(tokens) > max_seq_len - special_tokens_count:
        tokens = tokens[: (max_seq_len - special_tokens_count)]
        slot_label_mask = slot_label_mask[:(
            max_seq_len - special_tokens_count)]
        # !!!!!!!!!!!!!!!!!!!!!!!!!!!
        pro_labels_ids = pro_labels_ids[:(
            max_seq_len - special_tokens_count)]

    # Add [SEP] token
    tokens += [sep_token]
    token_type_ids = [sequence_a_segment_id] * len(tokens)
    slot_label_mask += [pad_token_label_id]
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    pro_labels_ids += [pad_token_label_id]

    # Add [CLS] token
    tokens = [cls_token] + tokens
    token_type_ids = [cls_token_segment_id] + token_type_ids
    slot_label_mask = [pad_token_label_id] + slot_label_mask
    pro_labels_ids = [pad_token_label_id] + \
        pro_labels_ids  # !!!!!!!!!!!!!!!!!!!!!!!
    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real tokens are attended to.
    attention_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding_length = max_seq_len - len(input_ids)
    input_ids = input_ids + ([pad_token_id] * padding_length)
    attention_mask = attention_mask + \
        ([0 if mask_padding_with_zero else 1] * padding_length)
    token_type_ids = token_type_ids + \
        ([pad_token_segment_id] * padding_length)

    input_ids = np.array(input_ids).astype('int64')
    attention_mask = np.array(attention_mask).astype('int64')
    token_type_ids = np.array(token_type_ids).astype('int64')
    # sample = {'input_ids': input_ids[None, :], 'attention_mask': attention_mask[None,
    #                                                                             :], 'token_type_ids': token_type_ids[None, :]}

    if model_name == 'distil_bert':
        return input_ids[None, :], attention_mask[None, :]
        #sample = {'input_ids': input_ids[None, :], 'attention_mask': attention_mask[None,:]}

    return input_ids[None, :], attention_mask[None, :], token_type_ids[None, :]

In [ ]:
import tensorflow as tf
import numpy as np

def representative_dataset():
    # open seq.in file and read the first 100 lines
    model_name = 'mobile_bert'
    with open('/home/maelic/Documents/nlp/NLU_Benchmarking/gpsr_pro_instance_say/test/seq.in', 'r') as f:
        lines = f.read().splitlines()[:100]

    for line in lines:
        if model_name == 'distil_bert':
            input_ids, attention_masks = convert_input_file_to_dataloader(line, model_name)
        else:
            input_ids, attention_masks, token_type_ids = convert_input_file_to_dataloader(line, model_name)

    # input_ids = np.ones([10, 32], dtype=np.int64)
    # attention_masks = np.ones([10, 32], dtype=np.int64)
    # token_type_ids = np.ones([10, 32], dtype=np.int64)

    if model_name == 'distil_bert':
        for attention_mask, input_id in zip(attention_masks, input_ids):
            yield {
                "attention_mask": attention_mask,
                "input_ids": input_id,
            }
    else:

        for attention_mask, input_id, token_type_id \
            in zip(attention_masks, input_ids, token_type_ids):

            yield {
                "attention_mask": attention_mask,
                "input_ids": input_id,
                "token_type_ids": token_type_id,
            }

model_name = 'mobile_bert'
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()

with open('saved_model/int8_model.tflite', 'wb') as w:
    w.write(tflite_quant_model)